In [59]:
import json
from itertools import chain
from stix2 import MemoryStore
from stix2 import Filter, IntrusionSet, Relationship

In [9]:
src = MemoryStore()
src.load_from_file("enterprise-attack.json")

In [19]:
with open('NewIncident.json') as fh:
    data = json.load(fh)
incident_technique_ids = []
for tech in data['techniques']:
    incident_technique_ids.append(tech['techniqueID'])
incident_technique_ids = list(set(incident_technique_ids))

In [79]:
import os
type(os.path.abspath('enterprise-attack.json'))

str

In [85]:
pattern_ids = {}
for tech_id in incident_technique_ids:
    tmp = src.query([ 
        Filter("external_references.external_id", "=", tech_id), 
        Filter("type", "=", "attack-pattern")
    ])[0]
    
    pattern_ids[tmp.id] = tmp.name

In [92]:
tmp = src.query([ 
        Filter("external_references.external_id", "=", incident_technique_ids[0]), 
        Filter("type", "=", "attack-pattern")
    ])[0]

[KillChainPhase(kill_chain_name='mitre-attack', phase_name='reconnaissance')]

In [86]:
pattern_ids

{'attack-pattern--5502c4e9-24ef-4d5f-8ee9-9e906c2f82c4': 'Vulnerability Scanning',
 'attack-pattern--7385dfaf-6886-4229-9ecd-6fd678040830': 'Command and Scripting Interpreter',
 'attack-pattern--810d8072-afb6-4a56-9ee7-86379ac4a6f3': 'Botnet',
 'attack-pattern--b8902400-e6c5-4ba2-95aa-2d35b442b118': 'Encrypted Channel',
 'attack-pattern--57340c81-c025-4189-8fa0-fc7ede51bae4': 'Modify Registry',
 'attack-pattern--970a3432-3237-47ad-bcca-7d8cbb217736': 'PowerShell',
 'attack-pattern--a19e86f8-1c0a-4fea-8407-23b73d615776': 'Exfiltration Over Alternative Protocol',
 'attack-pattern--c21d5a77-d422-4a69-acd7-2c53c1faa34b': 'Non-Application Layer Protocol',
 'attack-pattern--bf176076-b789-408e-8cba-7275e81c0ada': 'Asymmetric Cryptography',
 'attack-pattern--21875073-b0ee-49e3-9077-1e2a885359af': 'Domain Account',
 'attack-pattern--30208d3e-0d6b-43c8-883e-44462a514619': 'Automated Collection',
 'attack-pattern--0a3ead4e-6d47-4ccb-854c-a6a4f9d96b22': 'OS Credential Dumping',
 'attack-pattern--b

In [39]:
def get_related(thesrc, src_type, rel_type, target_type, reverse=False):
    """build relationship mappings
       params:
         thesrc: MemoryStore to build relationship lookups for
         src_type: source type for the relationships, e.g "attack-pattern"
         rel_type: relationship type for the relationships, e.g "uses"
         target_type: target type for the relationship, e.g "intrusion-set"
         reverse: build reverse mapping of target to source
    """

    relationships = thesrc.query([
        Filter('type', '=', 'relationship'),
        Filter('relationship_type', '=', rel_type)
    ])

    # stix_id => [ { relationship, related_object_id } for each related object ]
    id_to_related = {} 

    # build the dict
    for relationship in relationships:
        if (src_type in relationship.source_ref and target_type in relationship.target_ref):
            if (relationship.source_ref in id_to_related and not reverse) or (relationship.target_ref in id_to_related and reverse):
                # append to existing entry
                if not reverse: 
                    id_to_related[relationship.source_ref].append({
                        "relationship": relationship,
                        "id": relationship.target_ref
                    })
                else: 
                    id_to_related[relationship.target_ref].append({
                        "relationship": relationship, 
                        "id": relationship.source_ref
                    })
            else: 
                # create a new entry
                if not reverse: 
                    id_to_related[relationship.source_ref] = [{
                        "relationship": relationship, 
                        "id": relationship.target_ref
                    }]
                else:
                    id_to_related[relationship.target_ref] = [{
                        "relationship": relationship, 
                        "id": relationship.source_ref
                    }]
    # all objects of relevant type
    if not reverse:
        targets = thesrc.query([
            Filter('type', '=', target_type),
        ])
    else:
        targets = thesrc.query([
            Filter('type', '=', src_type),
        ])
    
    # remove revoked and deprecated objects from output
    targets = list(
        filter(
            lambda x: x.get("x_mitre_deprecated", False) is False and x.get("revoked", False) is False,
            targets
        )
    )

    # build lookup of stixID to stix object
    id_to_target = {}
    for target in targets:
        id_to_target[target.id] = target

    # build final output mappings
    output = {}
    for stix_id in id_to_related:
        value = []
        for related in id_to_related[stix_id]:
            if not related["id"] in id_to_target:
                continue # targeting a revoked object
            value.append({
                "object": id_to_target[related["id"]],
                "relationship": related["relationship"]
            })
        output[stix_id] = value
    return output


def techniques_used_by_groups(thesrc):
    """returns group_id => {technique, relationship} for each technique used by the group."""
    return get_related(thesrc, "intrusion-set", "uses", "attack-pattern")

def groups_using_technique(thesrc):
    """returns technique_id => {group, relationship} for each group using the technique."""
    return get_related(thesrc, "intrusion-set", "uses", "attack-pattern", reverse=True)

In [95]:
techs = groups_using_technique(src)

In [97]:
def related_threat_actors(attack_patterns: dict, techs_to_groups: dict) -> list():
    related_groups = dict()
    groups_techs = dict()
    for pid in attack_patterns.keys():
        if pid in techs_to_groups.keys():
            all_groups_by_pid_tech = techs_to_groups[pid]
            for group in all_groups_by_pid_tech:
                group_name = group['object']['name']
                if group_name not in related_groups.keys():
                    related_groups[group_name] = 1
                    groups_techs[group_name] = [attack_patterns[pid]]
                else:
                    related_groups[group_name] += 1
                    groups_techs[group_name].append(attack_patterns[pid])
    related_groups = {k: v for k, v in sorted(related_groups.items(), key=lambda item: item[1], reverse=True)}
    return list(related_groups.items())[0:10], groups_techs

In [98]:
related_groups, groups_techs = related_threat_actors(pattern_ids, techs)

In [101]:
related_groups

[('APT28', 12),
 ('OilRig', 12),
 ('APT29', 11),
 ('FIN6', 10),
 ('APT39', 10),
 ('Operation Wocao', 10),
 ('Sandworm Team', 9),
 ('APT32', 9),
 ('BRONZE BUTLER', 9),
 ('Patchwork', 9)]

In [102]:
def software_used_by_groups(thesrc):
    """returns group_id => {software, relationship} for each software used by the group."""
    x = get_related(thesrc, "intrusion-set", "uses", "malware")
    x_tool = get_related(thesrc, "intrusion-set", "uses", "tool")
    for key in x_tool:
      if key in x:
        x[key].extend(x_tool[key])
      else:
        x[key] = x_tool[key]
    return x


In [105]:
def get_group_by_alias(thesrc, alias):
    return thesrc.query([
        Filter('type', '=', 'intrusion-set'),
        Filter('aliases', '=', alias)
    ])[0]

In [109]:
APT_aliases = [i[0] for i in related_groups]
groups_id = list([get_group_by_alias(src, alias).id for alias in APT_aliases])
groups_id_dict = {}
for i in range(len(APT_aliases)):
    groups_id_dict[groups_id[i]] = APT_aliases[i]
groups_id_dict

{'intrusion-set--bef4c620-0787-42a8-a96d-b7eb6e85917c': 'APT28',
 'intrusion-set--4ca1929c-7d64-4aab-b849-badbfc0c760d': 'OilRig',
 'intrusion-set--899ce53f-13a0-479b-a0e4-67d46e241542': 'APT29',
 'intrusion-set--2a7914cf-dff3-428d-ab0f-1014d1c28aeb': 'FIN6',
 'intrusion-set--44e43fad-ffcb-4210-abcf-eaaed9735f80': 'APT39',
 'intrusion-set--28f04ed3-8e91-4805-b1f6-869020517871': 'Operation Wocao',
 'intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc35192': 'Sandworm Team',
 'intrusion-set--247cb30b-955f-42eb-97a5-a89fef69341e': 'APT32',
 'intrusion-set--93f52415-0fe4-4d3d-896c-fc9b8e88ab90': 'BRONZE BUTLER',
 'intrusion-set--17862c7d-9e60-48a0-b48e-da4dc4c3f6b0': 'Patchwork'}

In [103]:
group_soft = software_used_by_groups(src)

In [125]:
group_soft['intrusion-set--0bbdf25b-30ff-4894-a1cd-49260d0dd2d9'][0]['object']

Malware(type='malware', spec_version='2.1', id='malware--4e6b9625-bbda-4d96-a652-b3bb45453f26', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2018-01-16T16:13:52.465Z', modified='2020-03-31T12:40:01.208Z', name='RemoteCMD', description="[RemoteCMD](https://attack.mitre.org/software/S0166) is a custom tool used by [APT3](https://attack.mitre.org/groups/G0022) to execute commands on a remote system similar to SysInternal's PSEXEC functionality. (Citation: Symantec Buckeye)", is_family=True, revoked=False, external_references=[ExternalReference(source_name='mitre-attack', url='https://attack.mitre.org/software/S0166', external_id='S0166'), ExternalReference(source_name='Symantec Buckeye', description='Symantec Security Response. (2016, September 6). Buckeye cyberespionage group shifts gaze from US to Hong Kong. Retrieved September 26, 2016.', url='http://www.symantec.com/connect/blogs/buckeye-cyberespionage-group-shifts-gaze-us-hong-kong')], object_marking_refs

In [133]:
def related_soft(intrusion_sets: dict, groups_to_soft: dict) -> list():
    related_soft = list()
    groups_soft = dict()
    for isid in intrusion_sets.keys():
        all_groups_by_isid_soft = groups_to_soft[isid]
        for soft in all_groups_by_isid_soft:
            soft_info = (soft['object']['type'], soft['object']['name'], soft['object'].external_references, soft['object']['description'])
            if soft_info[1] not in related_soft:
                related_soft.append(soft_info)
            if intrusion_sets[isid] not in groups_soft.keys():
                groups_soft[intrusion_sets[isid]] = [soft_info[1]]
            elif soft_info[1] not in  groups_soft[intrusion_sets[isid]]:
                groups_soft[intrusion_sets[isid]].append(soft_info[1])
    # related_groups = {k: v for k, v in sorted(related_groups.items(), key=lambda item: item[1], reverse=True)}
    return related_soft, groups_soft

In [134]:
soft, soft_map = related_soft(groups_id_dict, group_soft)
soft_map

{'APT28': ['USBStealer',
  'HIDEDRV',
  'JHUHUGIT',
  'DealersChoice',
  'ADVSTORESHELL',
  'OLDBAIT',
  'XAgentOSX',
  'XTunnel',
  'Zebrocy',
  'CORESHELL',
  'Cannon',
  'CHOPSTICK',
  'Komplex',
  'LoJax',
  'Downdelph',
  'Fysbis',
  'Drovorub',
  'Mimikatz',
  'Responder',
  'certutil',
  'Winexe',
  'Koadic',
  'Forfiles',
  'Net',
  'Tor',
  'Wevtutil'],
 'OilRig': ['RGDoor',
  'SEASHARPEE',
  'OopsIE',
  'POWRUNER',
  'Helminth',
  'ISMInjector',
  'QUADAGENT',
  'BONDUPDATER',
  'RDAT',
  'SideTwist',
  'Mimikatz',
  'netstat',
  'Tasklist',
  'Reg',
  'PsExec',
  'certutil',
  'LaZagne',
  'Systeminfo',
  'ipconfig',
  'Net',
  'FTP'],
 'APT29': ['PinchDuke',
  'CozyCar',
  'CloudDuke',
  'PowerDuke',
  'GeminiDuke',
  'CosmicDuke',
  'MiniDuke',
  'HAMMERTOSS',
  'POSHSPY',
  'OnionDuke',
  'SeaDuke',
  'PolyglotDuke',
  'RegDuke',
  'WellMess',
  'WellMail',
  'SoreFang',
  'Cobalt Strike',
  'SUNSPOT',
  'Sibot',
  'TEARDROP',
  'Raindrop',
  'GoldFinder',
  'GoldMax',
  

In [135]:
soft

[('malware',
  'USBStealer',
  [ExternalReference(source_name='mitre-attack', url='https://attack.mitre.org/software/S0136', external_id='S0136'),
   ExternalReference(source_name='ESET Sednit USBStealer 2014', description='Calvet, J. (2014, November 11). Sednit Espionage Group Attacking Air-Gapped Networks. Retrieved January 4, 2017.', url='http://www.welivesecurity.com/2014/11/11/sednit-espionage-group-attacking-air-gapped-networks/'),
   ExternalReference(source_name='Kaspersky Sofacy', description="Kaspersky Lab's Global Research and Analysis Team. (2015, December 4). Sofacy APT hits high profile targets with updated toolset. Retrieved December 10, 2015.", url='https://securelist.com/sofacy-apt-hits-high-profile-targets-with-updated-toolset/72924/')],
  '[USBStealer](https://attack.mitre.org/software/S0136) is malware that has used by [APT28](https://attack.mitre.org/groups/G0007) since at least 2005 to extract information from air-gapped networks. It does not have the capability t